In [1]:
USE_H5PY = False  # set to True to use h5py/hdf5lib instead
if USE_H5PY:
    import h5py
    from h5py import MultiManager
else:
    import h5pyd as h5py  # Use the "as" syntax for code  compatibility
    from h5pyd import MultiManager
import numpy as np

In [2]:
# create a new file
f = h5py.File("/home/test_user1/multi_try.h5", mode="w")

In [3]:
# create some datasets
DSET_SHAPE = (10,)
DSET_DTYPE = np.int32

# create 4 datasets
DSET_COUNT = 4
datasets = []
for i in range(DSET_COUNT):
    dset = f.create_dataset(f"dset_{i}", shape=DSET_SHAPE, dtype=DSET_DTYPE)
    datasets.append(dset)


In [4]:
# initialize some data to write
data_in = []
for n in range(DSET_COUNT):
    arr = np.zeros(DSET_SHAPE, dtype=DSET_DTYPE)
    arr[...] = list(range(n*100, n*100+DSET_SHAPE[0]))
    data_in.append(arr)
    print(f"data_in[{n}]: {arr}")


data_in[0]: [0 1 2 3 4 5 6 7 8 9]
data_in[1]: [100 101 102 103 104 105 106 107 108 109]
data_in[2]: [200 201 202 203 204 205 206 207 208 209]
data_in[3]: [300 301 302 303 304 305 306 307 308 309]


In [5]:
# instantiate a MultiManager and use it to write to all the datasets simultaneously
mm = MultiManager(datasets)
mm[...] = data_in

In [6]:
# verify what get saved to each dataset
for n in range(DSET_COUNT):
    dset_name = f"dset_{n}"
    dset = f[dset_name]
    print(f"{dset_name}: {dset[...]}")

dset_0: [0 1 2 3 4 5 6 7 8 9]
dset_1: [100 101 102 103 104 105 106 107 108 109]
dset_2: [200 201 202 203 204 205 206 207 208 209]
dset_3: [300 301 302 303 304 305 306 307 308 309]


In [7]:
# Read all the data from all the daasets using the same MultiManager instance
data_out = mm[...]
len(data_out)

4

In [8]:
# dump the data that was returned
for n in range(DSET_COUNT):
    print(f"data_out[{n}]: {data_out[n]}")

data_out[0]: [0 1 2 3 4 5 6 7 8 9]
data_out[1]: [100 101 102 103 104 105 106 107 108 109]
data_out[2]: [200 201 202 203 204 205 206 207 208 209]
data_out[3]: [300 301 302 303 304 305 306 307 308 309]


In [9]:
# rather than reading all the data for a dataset, you can read a given selection
data_out = mm[0:4]
for n in range(DSET_COUNT):
    print(f"data_out[{n}]: {data_out[n]}")

data_out[0]: [0 1 2 3]
data_out[1]: [100 101 102 103]
data_out[2]: [200 201 202 203]
data_out[3]: [300 301 302 303]


In [10]:
# it's also possible to pass a list of selections and have each selection
# read from the corresponding dataset
selections = []
for n in range(DSET_COUNT):
    s = slice(n, n+2, 1)
    selections.append(s)

data_out = mm.__getitem__(selections)

In [11]:
# dump the data that was returned
for n in range(DSET_COUNT):
    print(f"data_out[{n}]: {data_out[n]}")

data_out[0]: [0 1]
data_out[1]: [101 102]
data_out[2]: [202 203]
data_out[3]: [303 304]
